In [ ]:
# import modules
from twitter import *
from urllib.error import HTTPError
import pandas as pd
import json
import time

In [ ]:
# temp util
def timestamp_attrs(data, col='created_at', tz='America/Bogota', passtz=True):
    '''
    '''
    if passtz:
        t = pd.to_datetime(
            data[col],
            utc=True,
            infer_datetime_format=True
        ).apply(lambda x: x.tz_convert(tz))
    else:
        t = pd.to_datetime(
            data[col],
            infer_datetime_format=True
        )

    data[f'{col}_timestamp'] = t
    data[f'{col}'] = t.dt.strftime('%Y-%m-%d')
    data[f'{col}_year'] = t.dt.year
    data[f'{col}_month'] = t.dt.month
    data[f'{col}_day'] = t.dt.day
    data[f'{col}_hour'] = t.dt.hour
    data[f'{col}_weekday'] = t.dt.dayofweek
    data[f'{col}_time_hour'] = t.dt.strftime('%H:%M:%S')
    
    return data

**`Twitter Auth`**

In [ ]:
# read credentials
with open('../_config/twitter_auth.json') as f:
    auth = json.load(f)
    f.close()

# Auth
Auth = OAuth(**auth)

# connection
cxn = Twitter(auth=Auth)

**`Get followers`**

In [ ]:
# get followers
def get_followers(username, count=5000, get_all=True):
    '''
    '''
    params = {
        'screen_name': username,
        'cursor': -1,
        'stringify_ids': True,
        'count': count
    }
    
    # status requests
    status = cxn.application.rate_limit_status()
    
    # process users
    users = cxn.followers.ids(**params)
    followers = users['ids']
    cursor = users['next_cursor']
    
    # write user ids
    writer_path = 'C:/i/repositories/DigitalResearch/data/uribe_dcigroup/followers.txt'
    writer = open(writer_path, mode='w')
    for u in followers:
        writer.write(f'{u}\n')
    
    writer.close()
    
    # lookup
    json_path = 'C:/i/repositories/DigitalResearch/data/uribe_dcigroup/followers/followers{}.json'
    serie = 0
    batch = 100
    ids = [followers[i: i + batch] for i in range(0, len(followers), batch)]
    
    lookup_remaining_calls = status['resources']['users']['/users/lookup']['remaining']
    for accounts in ids:
        if lookup_remaining_calls == 50:
            status = cnx.application.rate_limit_status()
            lookup_remaining_calls = status['resources']['users']['/users/lookup']['remaining']
        
        if lookup_remaining_calls <= 0:
            sleep_until = status['resources']['users']['/users/lookup']['reset']
            sleep_for = int(sleep_until - time.time()) + 10
            if sleep_for > 0:
                print (f'Lockup users -> Sleeping for {sleep_for} seconds...')
                time.sleep(sleep_for)
                print ('Awake!')
                print ('')
                
                status = cnx.application.rate_limit_status()
                lookup_remaining_calls = status['resources']['users']['/users/lookup']['remaining']
        
        lookup_remaining_calls -= 1
        
        # 100 users per request
        users_lookup = cxn.users.lookup(user_id=','.join(accounts))
        
        # write bullk data
        _json_path_ = json_path.format(serie)
        Obj = json.dumps(users_lookup, ensure_ascii=False, separators=(',',':'))
        json_writer = open(_json_path_, mode='w', encoding='utf-8')
        json_writer.write(Obj)
        json_writer.close()
        
        # increment 1
        serie += 1

    # iterate
    if get_all != True:
        return
    else:
        while cursor != 0:
            try:
                params['cursor'] = cursor
                users = cxn.followers.ids(**params)
                followers = users['ids']
                cursor = users['next_cursor']
                
                # append new followers
                writer = open(writer_path, mode='a')
                for u in followers:
                    writer.write(f'{u}\n')

                writer.close()
                
                # batch
                ids = [followers[i: i + batch] for i in range(0, len(followers), batch)]
                
                lookup_remaining_calls = status['resources']['users']['/users/lookup']['remaining']
                
                for accounts in ids:
                    if lookup_remaining_calls == 50:
                        status = cnx.application.rate_limit_status()
                        lookup_remaining_calls = status['resources']['users']['/users/lookup']['remaining']

                    if lookup_remaining_calls <= 0:
                        sleep_until = status['resources']['users']['/users/lookup']['reset']
                        sleep_for = int(sleep_until - time.time()) + 10
                        if sleep_for > 0:
                            print (f'Lockup users -> Sleeping for {sleep_for} seconds...')
                            time.sleep(sleep_for)
                            print ('Awake!')
                            print ('')

                            status = cnx.application.rate_limit_status()
                            lookup_remaining_calls = status['resources']['users']['/users/lookup']['remaining']

                    lookup_remaining_calls -= 1

                    # 100 users per request
                    users_lookup = cxn.users.lookup(user_id=','.join(accounts))

                    # write bullk data
                    _json_path_ = json_path.format(serie)
                    Obj = json.dumps(users_lookup, ensure_ascii=False, separators=(',',':'))
                    json_writer = open(_json_path_, mode='w', encoding='utf-8')
                    json_writer.write(Obj)
                    json_writer.close()

                    # increment 1
                    serie += 1
            
            except (TwitterHTTPError, HTTPError):
                
                remain = status['resources']['followers']['/followers/ids']['remaining']
                reset = status['resources']['followers']['/followers/ids']['reset']
                
                if remain <= 10:
                    sleep_for = int(reset - time.time()) + 10
                    print (f'Getting followers -> Sleeping for {sleep_for} seconds...')
                    time.sleep(sleep_for)
                    print ('Awake!')
                    print ('')
                
                continue
        
        return

In [ ]:
username = 'FreeUribe'
get_followers(username)

_Explore followers_

In [ ]:
division = 100
ids = [followers[i: i + division] for i in range(0, len(followers), division)]
all_ = []
for accounts in ids:
    users_lookup = cxn.users.lookup(user_id=','.join(accounts))
    all_.extend(users_lookup)
    
print (f'Metadata -> {len(all_)}')

In [ ]:
df = pd.DataFrame.from_dict(all_)
df.shape

In [ ]:
df.to_excel('../_data/accounts.xlsx', index=False)

In [ ]:
df.head()

_load columns order_

In [ ]:
order = [i.rstrip() for i in open('../assets/get_followers/columns_order.txt', encoding='utf-8')]

_create dataset_

In [ ]:
dataset = timestamp_attrs(df, tz='America/New_York')

In [ ]:
dataset = dataset[order].copy()

In [ ]:
dataset['created_at_timestamp'] = dataset['created_at_timestamp'].astype(str)

In [ ]:
path = f'../_data/followers_resilience/{username}_followers.xlsx'
dataset.to_excel(path, index=False)

In [ ]:
df[['screen_name', 'created_at']]